In [5]:
import os
import numpy as np
from PIL import Image
from sklearn.metrics import precision_score, recall_score, f1_score, jaccard_score
import cv2
import re
# import os
import glob
# import re
# import numpy as np
# from sklearn.metrics import jaccard_score, precision_score, recall_score, f1_score
import tifffile as tiff
# import os
# import numpy as np
import glob
# from sklearn.metrics import jaccard_score, precision_score, recall_score, f1_score
# import rasterio

import json

# with open('config.json') as json_file:
#     config = json.load(json_file)
#     corruption = config['corruption_level']


def load_image(filepath):
    # Load the image using PIL
    image = np.array(Image.open(filepath).convert('L'))
    
    # Apply Otsu's thresholding
    t, binary_image = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    # Convert to binary (0 and 1)
    binary_image = (binary_image > 0).astype(np.uint8)
    return binary_image

def dice_coefficient(y_true, y_pred):
    intersection = np.sum(y_true * y_pred)
    return (2. * intersection) / (np.sum(y_true) + np.sum(y_pred))

def pixel_accuracy(y_true, y_pred):
    return np.mean(y_true == y_pred)

def specificity(y_true, y_pred):
    tn = np.sum((y_true == 0) & (y_pred == 0))
    fp = np.sum((y_true == 0) & (y_pred == 1))
    return tn / (tn + fp)

# Paths to the folders containing the predictions and ground truths


In [6]:
def dice_coefficient(y_true, y_pred, smooth=1):
    y_true_f = y_true.flatten()
    y_pred_f = y_pred.flatten()
    intersection = np.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (np.sum(y_true_f) + np.sum(y_pred_f) + smooth)

def pixel_accuracy(y_true, y_pred):
    return np.mean(y_true == y_pred)

def specificity(y_true, y_pred):
    tn = np.sum((y_true == 0) & (y_pred == 0))
    fp = np.sum((y_true == 0) & (y_pred == 1))
    return tn / (tn + fp)


# Directories for ground truth and prediction masks (Unet)
# gt_folder = './GEE_Masks/GEE_resized/'
# Directories for ground truth masks (DeepLabV3, Segformer, Maskformer)
gt_folder = './sar_images/masks/test'


# For UNet
# pred_folder = f'./GEE_Output/Adversarial/New_20_Epoch_{corruption}_with_diff_kernels'
# pred_files = glob.glob(os.path.join(pred_folder, 'dense_*.tif'))

# For DeepLabV3
pred_folder = f'./GEE_Output/Segformer/Outputs'

# For Segformer
# pred_folder = f'./GEE_Output/Segformer/Outputs'

# For Maskformer
# pred_folder = f'./GEE_Output/Maskformer/Outputs'

print(pred_folder)

# Initialize metric lists
dice_scores = []
iou_scores = []
pixel_accuracies = []
precisions = []
recalls = []
f1_scores = []
specificities = []

# Get all prediction files

# pred_files = glob.glob(os.path.join(pred_folder, 'mobilenetv2_*.tif'))
# For DeepLabV3
pred_files = glob.glob(os.path.join(pred_folder, '*.png'))

for pred_file in pred_files:
    # Extract the number 'i' from the prediction filename
    # For UNet
    # match = re.search(r'dense_(\d+).tif', os.path.basename(pred_file))
    # For DeeplabV3, Segformer, Maskformer
    match = re.search(r'pred_(\d+)\.png', os.path.basename(pred_file))
    
    if match:
        i = match.group(1)
        
        # For UNet
        # gt_filename = f'NDWI_Mask_{i}_resized.tif'
        
        # For Deeplabv3, Segformer and Maskformer
        gt_filename = f'{i}.png'
        
        gt_path = os.path.join(gt_folder, gt_filename)
        
        # Check if the ground truth file exists
        if os.path.exists(gt_path):
            # Load the prediction and ground truth images
            pred_image = load_image(pred_file)
            gt_image = load_image(gt_path)
            print(np.max(pred_image))
            print(np.min(pred_image))
            # Flatten the images
            pred_flat = pred_image.flatten()
            gt_flat = gt_image.flatten()
            
            # Calculate metrics
            dice = dice_coefficient(gt_flat, pred_flat)
            iou = jaccard_score(gt_flat, pred_flat, average='macro')
            accuracy = pixel_accuracy(gt_flat, pred_flat)
            precision = precision_score(gt_flat, pred_flat, average='macro', zero_division=0)
            recall = recall_score(gt_flat, pred_flat, average='macro')
            f1 = f1_score(gt_flat, pred_flat, average='macro')
            spec = specificity(gt_flat, pred_flat)
            
            # Store metrics
            dice_scores.append(dice)
            iou_scores.append(iou)
            pixel_accuracies.append(accuracy)
            precisions.append(precision)
            recalls.append(recall)
            f1_scores.append(f1)
            specificities.append(spec)
            
            # Print metrics for the current image
            print(f"Metrics for {gt_filename} and {pred_file}:")
            print(f"  Dice Coefficient: {dice:.4f}")
            print(f"  IoU: {iou:.4f}")
            print(f"  Pixel Accuracy: {accuracy:.4f}")
            print(f"  Precision: {precision:.4f}")
            print(f"  Recall: {recall:.4f}")
            print(f"  F1 Score: {f1:.4f}")
            print(f"  Specificity: {spec:.4f}")
            print()
        else:
            print(f"Ground truth file {gt_path} not found for prediction file {pred_file}.")

# Calculate and print average metrics
average_dice = np.mean(dice_scores)
average_iou = np.mean(iou_scores)
average_accuracy = np.mean(pixel_accuracies)
average_precision = np.mean(precisions)
average_recall = np.mean(recalls)
average_f1 = np.mean(f1_scores)
average_specificity = np.mean(specificities)

print("Average Metrics:")
print(f"  Dice Coefficient: {average_dice:.4f}")
print(f"  IoU: {average_iou:.4f}")
print(f"  Pixel Accuracy: {average_accuracy:.4f}")
print(f"  Precision: {average_precision:.4f}")
print(f"  Recall: {average_recall:.4f}")
print(f"  F1 Score: {average_f1:.4f}")
print(f"  Specificity: {average_specificity:.4f}")

./GEE_Output/Segformer/Outputs
1
0
Metrics for 10.png and ./GEE_Output/Segformer/Outputs\pred_10.png:
  Dice Coefficient: 0.6892
  IoU: 0.6419
  Pixel Accuracy: 0.8092
  Precision: 0.8655
  Recall: 0.7617
  F1 Score: 0.7758
  Specificity: 0.9876

1
0
Metrics for 1008.png and ./GEE_Output/Segformer/Outputs\pred_1008.png:
  Dice Coefficient: 0.8663
  IoU: 0.8717
  Pixel Accuracy: 0.9806
  Precision: 0.9778
  Recall: 0.8889
  F1 Score: 0.9279
  Specificity: 0.9982

1
0
Metrics for 1010.png and ./GEE_Output/Segformer/Outputs\pred_1010.png:
  Dice Coefficient: 0.9901
  IoU: 0.9852
  Pixel Accuracy: 0.9933
  Precision: 0.9950
  Recall: 0.9902
  F1 Score: 0.9925
  Specificity: 0.9999

1
0
Metrics for 1012.png and ./GEE_Output/Segformer/Outputs\pred_1012.png:
  Dice Coefficient: 0.8807
  IoU: 0.8862
  Pixel Accuracy: 0.9862
  Precision: 0.9769
  Recall: 0.9035
  F1 Score: 0.9367
  Specificity: 0.9981

1
0
Metrics for 1020.png and ./GEE_Output/Segformer/Outputs\pred_1020.png:
  Dice Coefficient

c:\Users\ADMIN\anaconda3\envs\tf_gpu\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Metrics for 814.png and ./GEE_Output/Segformer/Outputs\pred_814.png:
  Dice Coefficient: 0.7904
  IoU: 0.7716
  Pixel Accuracy: 0.9087
  Precision: 0.8402
  Recall: 0.9042
  F1 Score: 0.8660
  Specificity: 0.9115

1
0
Metrics for 837.png and ./GEE_Output/Segformer/Outputs\pred_837.png:
  Dice Coefficient: 0.7397
  IoU: 0.7236
  Pixel Accuracy: 0.8834
  Precision: 0.7992
  Recall: 0.8985
  F1 Score: 0.8323
  Specificity: 0.8748

1
0
Metrics for 838.png and ./GEE_Output/Segformer/Outputs\pred_838.png:
  Dice Coefficient: 0.3702
  IoU: 0.5476
  Pixel Accuracy: 0.8730
  Precision: 0.6170
  Recall: 0.7787
  F1 Score: 0.6498
  Specificity: 0.8847

1
0
Metrics for 846.png and ./GEE_Output/Segformer/Outputs\pred_846.png:
  Dice Coefficient: 0.5179
  IoU: 0.6521
  Pixel Accuracy: 0.9558
  Precision: 0.9641
  Recall: 0.6761
  F1 Score: 0.7474
  Specificity: 0.9993

1
0
Metrics for 847.png and ./GEE_Output/Segformer/Outputs\pred_847.png:
  Dice Coefficient: 0.4921
  IoU: 0.6170
  Pixel Accuracy: 

In [7]:
print("Average metrics:")
print(f"  Average Dice Coefficient: {average_dice:.4f}")
print(f"  Average IoU: {average_iou:.4f}")
print(f"  Average Pixel Accuracy: {average_accuracy:.4f}")
print(f"  Average Precision: {average_precision:.4f}")
print(f"  Average Recall: {average_recall:.4f}")
print(f"  Average F1 Score: {average_f1:.4f}")
print(f"  Average Specificity: {average_specificity:.4f}")

# Model's overall accuracy
print(f"Model's Overall Accuracy: {average_accuracy:.4f}")

Average metrics:
  Average Dice Coefficient: 0.7770
  Average IoU: 0.8291
  Average Pixel Accuracy: 0.9566
  Average Precision: 0.9079
  Average Recall: 0.8678
  Average F1 Score: 0.8701
  Average Specificity: 0.9746
Model's Overall Accuracy: 0.9566
